# Working with BIDS Data

In [17]:
import matplotlib
import pathlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

### Read the input data we wish to convert to BIDS

In [18]:
sample_data_dir = mne.datasets.sample.data_path()
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

Opening raw data file C:\Users\hp\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
320 events found
Event IDs: [ 1  2  3  4  5 32]


### Write the Raw data to BIDS!

In [16]:
raw.info['line_freq'] = 60 ## Setting the power line frequency to 60 Hz.

subject_info = {
    'birthday': (1988, 10, 1),
    'sex': 2,
    'hand': 3
}

raw.info['subject_info'] = subject_info
raw.info

<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 GRAD, 102 MAG, 9 STIM, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 line_freq: 60
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: off, PCA-v2: off, PCA-v3: off
 sfreq: 600.6 Hz
 subject_info: 3 items (dict)
>

In [11]:
out_path = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '02',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             root = out_path)

mne_bids.write_raw_bids(raw, bids_path = bids_path, events_data = events, event_id = event_id, overwrite = True)

Opening raw data file C:\Users\hp\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.

Writing 'out_data\sample_BIDS\participants.tsv'...

participant_id	age	sex	hand
sub-01	14	F	A
sub-02	n/a	n/a	n/a

Writing 'out_data\sample_BIDS\participants.json'...

{
    "participant_id": {
        "Description": "Unique participant identifier"
    },
    "age": {
        "Description": "Age of the participant at time of testing",
        "Units": "years"
    },
    "sex": {
        "Description": "Biological sex of the participant",
        "Levels": {
            "F": "female",
            "M": "male"
        }
    },
    "hand": {
        "Description": "Handedness of the participant",
        "Levels": {
            "R": "right",
            "L": "left",
            "A": "ambidextrous"
 

BIDSPath(
root: out_data\sample_BIDS
datatype: meg
basename: sub-02_ses-01_task-audiovisual_run-01_meg.fif)

### Write MEGIN / Elekta / NeuroMag fine-calibration and crosstalk files

In [12]:
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path = bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path = bids_path)

Writing fine-calibration file to out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_acq-calibration_meg.dat
Writing crosstalk file to out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_acq-crosstalk_meg.fif


### Print the created file and directory structure

In [13]:
mne_bids.print_dir_tree(out_path)

|sample_BIDS\
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01\
|------ ses-01\
|--------- sub-01_ses-01_scans.tsv
|--------- meg\
|------------ sub-01_ses-01_acq-calibration_meg.dat
|------------ sub-01_ses-01_acq-crosstalk_meg.fif
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.fif
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.json
|--- sub-02\
|------ ses-01\
|--------- sub-02_ses-01_scans.tsv
|--------- meg\
|------------ sub-02_ses-01_acq-calibration_meg.dat
|------------ sub-02_ses-01_acq-crosstalk_meg.fif
|------------ sub-02_ses-01_coordsystem.json
|------------ sub-02_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_meg.fif
|-------

### Generate a data summary

In [14]:
print(mne_bids.make_report(out_path))

Summarizing participants.tsv out_data\sample_BIDS\participants.tsv...
Summarizing scans.tsv files [WindowsPath('out_data/sample_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv'), WindowsPath('out_data/sample_BIDS/sub-02/ses-01/sub-02_ses-01_scans.tsv')]...
The participant template found: comprised of 0 men and 1 women;
comprised of 0 right hand, 0 left hand and 1 ambidextrous; ages ranged from 14.0 to 14.0 (mean = 14.0, std = 0.0; 1 with unknown age)
This dataset was created with BIDS version 1.4.0 by Please cite MNE-BIDS in your
publication before removing this (citations in README). This report was
generated with MNE-BIDS (https://doi.org/10.21105/joss.01896). The dataset
consists of 2 participants (comprised of 0 men and 1 women; comprised of 0 right
hand, 0 left hand and 1 ambidextrous; ages ranged from 14.0 to 14.0 (mean =
14.0, std = 0.0; 1 with unknown age))and 1 recording sessions: 01. Data was
recorded using a MEG system (Elekta manufacturer) sampled at 600.61 Hz with line
noise at

### Read BIDS data

In [19]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)
raw = mne_bids.read_raw_bids(bids_path)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.


C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


In [22]:
raw.plot()

<MNEBrowseFigure size 1920x940 with 5 Axes>

Channels marked as bad: ['MEG 2443', 'EEG 053']


### Events are stored as annotations - but we convert between the two.

In [23]:
raw.annotations[0]

OrderedDict([('onset', 46.58058882905339),
             ('duration', 0.0),
             ('description', 'Auditory/Right'),
             ('orig_time',
              datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc))])

In [24]:
raw.annotations[-1]

OrderedDict([('onset', 280.8321518290534),
             ('duration', 0.0),
             ('description', 'Button'),
             ('orig_time',
              datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc))])

In [25]:
mne.events_from_annotations?

Signature:
mne.events_from_annotations(
    raw,
    event_id='auto',
    regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$',
    use_rounding=True,
    chunk_duration=None,
    verbose=None,
)
Docstring:
Get events and event_id from an Annotations object.

Parameters
----------
raw : instance of Raw
    The raw data for which Annotations are defined.
event_id : dict | callable | None | 'auto'
    Can be:

    - **dict**: map descriptions (keys) to integer event codes (values).
      Only the descriptions present will be mapped, others will be ignored.
    - **callable**: must take a string input and return an integer event
      code, or return ``None`` to ignore the event.
    - **None**: Map descriptions to unique integer values based on their
      ``sorted`` order.
    - **'auto' (default)**: prefer a raw-format-specific parser:

      - Brainvision: map stimulus events to their integer part; response
        events to integer part + 1000; optic events to integer part + 2000;
        

In [26]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [29]:
events.shape

(320, 3)

In [27]:
mne.viz.plot_events(events, event_id = event_id, sfreq = raw.info['sfreq'])

<Figure size 640x476 with 1 Axes>

### Fine the MEG fine-calibration and crosstalk files

In [30]:
bids_path.meg_calibration_fpath

WindowsPath('out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-calibration_meg.dat')

In [31]:
bids_path.meg_crosstalk_fpath

WindowsPath('out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-crosstalk_meg.fif')